# Download and visualize urban data

In [ ]:
import sys
sys.path.append('..')
import urbanpy as up
import warnings
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm.auto import tqdm

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
tqdm.pandas()

## Downloading data for different cities and plotting

In [ ]:
ba = up.download.nominatim_osm('Buenos Aires, Argentina') # expected_position is 0 by default

In [ ]:
lima = up.download.nominatim_osm('Lima Metropolitana, Peru') # third result

In [ ]:
quito = up.download.nominatim_osm('Quito, Ecuador', 1) # second result

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,5))

ba.plot(ax=ax1)
lima.plot(ax=ax2)
quito.plot(ax=ax3)

plt.tight_layout()

### Downloading population data

In [ ]:
pop_arg = up.download.hdx_fb_population('argentina', 'full')

In [ ]:
# pop_ecu = up.download.hdx_fb_population('ecuador', 'full')
pop_ecu = up.download.hdx_dataset('58c3ac3f-febd-4222-8969-59c0fe0e7a0d/resource/c05a3c81-a78c-4e6c-ac05-de1316d4ba12/download/ecu_general_2020_csv.zip')

In [ ]:
pop_per = up.download.hdx_fb_population('peru', 'full')

In [ ]:
pop_per.head()

### Conversion to Point geometries and hexagons

We got the lat lon coordinates but their are just in numerical format, we need them as geometries to perform spatial operations on the hexagon grid

First, lets filter the national population to our city bounds

In [ ]:
pop_ba = up.geom.filter_population(pop_arg, ba)

In [ ]:
pop_quito = up.geom.filter_population(pop_ecu, quito)

In [ ]:
pop_lima = up.geom.filter_population(pop_per, lima)

In [ ]:
fig1, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,5))

pop_lima.plot(ax=ax1)
pop_ba.plot(ax=ax2)
pop_quito.plot(ax=ax3)

plt.tight_layout()

Applying filter_population gives us the set of points within our city's bounds and a Point geometry to work with hexagons

### Removing unnecesary features

We don't need the San Lorenzo island. By providing a bounding box we can remove it

In [ ]:
pop_lima = up.geom.remove_features(pop_lima, [-77.3, -12.2, -77.17, -12])

In [ ]:
pop_lima.plot()

Let's generate the hexagon grid using Uber's H3

In [ ]:
hex_ba = up.geom.gen_hexagons(9, ba)
hex_ba.shape

In [ ]:
hex_lima = up.geom.gen_hexagons(8, lima)
hex_lima.shape

In [ ]:
hex_quito = up.geom.gen_hexagons(7, quito)
hex_quito.shape

Notice the effect of the resolution parameter (higher resolution values generate smaller hexagons)

### Merging a layer

We got both the population point geometries and hexagons, lets get the population per hexagon.

In [ ]:
hex_ba = up.geom.merge_shape_hex(hex_ba, pop_ba, agg={'population_2020':'sum', 'population_2015':'mean'})

In [ ]:
hex_quito = up.geom.merge_shape_hex(hex_quito, pop_quito, agg={'ecu_general_2020':'sum'})

In [ ]:
hex_lima = up.geom.merge_shape_hex(hex_lima, pop_lima, agg={'population_2020':'sum', 'population_2015':'mean'})

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,5))

divider1 = make_axes_locatable(ax1)
cax1 = divider1.append_axes("right", size="5%", pad=0.1)
hex_quito.plot('ecu_general_2020', legend=True, missing_kwds={'color':'grey'}, ax=ax1, cax=cax1)
ax1.set_title('Quito - H3 res: 7')

divider2 = make_axes_locatable(ax2)
cax2 = divider2.append_axes("right", size="5%", pad=0.1)
hex_lima.plot('population_2020', legend=True, missing_kwds={'color':'grey'}, ax=ax2, cax=cax2)
ax2.set_title('Lima - H3 res: 8')

divider3 = make_axes_locatable(ax3)
cax3 = divider3.append_axes("right", size="5%", pad=0.1)
hex_ba.plot('population_2020', legend=True, missing_kwds={'color':'grey'}, ax=ax3, cax=cax3)
ax3.set_title('Buenos Aires - H3 res: 9')

plt.tight_layout()

We've used an inner join to keep only the points that intersect a hexagon (in this case, a within operation gives similar results). We provide a dictionary detailing how to aggregate our fields of interest

## Calculate trip distance and duration from each hexagon to the closest food facility

In [ ]:
hex_lima['lat'] = hex_lima.geometry.centroid.y
hex_lima['lon'] = hex_lima.geometry.centroid.x

In [ ]:
fs = up.download.overpass_pois(bounds=lima.total_bounds, facilities='food')

In [ ]:
fs.shape

In [ ]:
fs.plot()

Neighborhood search to find closest food facility

In [ ]:
dist_up, ind_up = up.utils.nn_search(
    tree_features=fs[['lat', 'lon']].values, # Point of Interest
    query_features=hex_lima[['lat', 'lon']].values, # Spatial Unit (Hexagons) Centroid
    metric='haversine' # Distance metric
)

In [ ]:
hex_lima['nearest_food_facility_ix'] = ind_up

Use OSRM routing server to find walking distance and duration 

In [ ]:
# start server
up.routing.start_osrm_server('peru', 'south-america', 'foot')

In [ ]:
# Distancia y duración del viaje a pie
distance_duration = hex_lima.progress_apply(
    lambda row: up.routing.osrm_route(
        origin=row.geometry.centroid, 
        destination = fs.iloc[row['nearest_food_facility_ix']]['geometry']
    ),
    result_type='expand',
    axis=1,
)

In [ ]:
hex_lima['distance_to_food_facility'] =  distance_duration[0] / 1000 # meters to km
hex_lima['duration_to_food_facility'] = distance_duration[1] / 60 # seconds to minutes

In [ ]:
up.routing.stop_osrm_server('peru', 'south-america', 'foot')

## Generate interactive maps

In [ ]:
fig = up.plotting.choropleth_map(hex_lima, 'population_2020', title='Estimated Population - 2020', 
                                 width=400, height=400)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.show()

Binarize durations to improve map visualization 

In [ ]:
custom_bins, custom_labels = up.utils.create_duration_labels(hex_lima['duration_to_food_facility'])

In [ ]:
hex_lima['duration_to_food_facility_bins'] = pd.cut(hex_lima['duration_to_food_facility'], bins=custom_bins, labels=custom_labels)

You can customize your plot usign the plotly.express.choropletmap function parameters

In [ ]:
fig = up.plotting.choropleth_map(
    hex_lima, 'duration_to_food_facility_bins',
    title='Acceso a instalaciones de venta de alimento',
    color_discrete_sequence=px.colors.sequential.Plasma_r, 
    category_orders={'duration_to_food_facility_bins': custom_labels}, 
    width=400, height=400
)

fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
)

fig.show()